
# Modelo Random Forest: Clasificación de Nivel de Contagios
## Paso 1: Definición del Problema y Preparación Inicial

### 🎯 Objetivo de la Actividad
En este primer capítulo, sentaremos las bases para construir un modelo de clasificación. Aprenderemos a definir correctamente nuestro problema de Machine Learning y a preparar el entorno y los datos para el análisis.

### 📋 Tareas de este capítulo:
1.  **Cargar las librerías** necesarias para el análisis de datos.
2.  **Importar el conjunto de datos** desde un archivo formato Stata (`.dta`).
3.  **Realizar una exploración inicial** para entender la dimensionalidad y estructura de los datos.
4.  **Crear la variable dependiente (objetivo)**, que es la variable que nuestro modelo intentará predecir.


### 1.1. Importación de Librerías

In [ ]:

import pandas as pd
import numpy as np
print("✅ Librerías importadas correctamente.")


### 1.2. Carga del Conjunto de Datos

In [ ]:

nombre_archivo = 'merge_RENAMU_GASTO_V.dta'
try:
    df = pd.read_stata(nombre_archivo)
    print(f"✅ Base de datos '{nombre_archivo}' cargada exitosamente.")
    print(f"📊 El conjunto de datos tiene {df.shape[0]} filas y {df.shape[1]} columnas.")
except FileNotFoundError:
    print(f"❌ Error: No se pudo encontrar el archivo '{nombre_archivo}'.")


### 1.3. Exploración Inicial de Datos

In [ ]:
df.head()

### 1.4. Creación de la Variable Dependiente (Objetivo)

In [ ]:

mediana_contagiados = df['contagiados'].median()
print(f"📊 La mediana de la variable 'contagiados' es: {mediana_contagiados}")

df['nivel_contagios'] = df['contagiados'].apply(lambda x: 'ALTO' if x > mediana_contagiados else 'BAJO')
print("\n✅ Columna 'nivel_contagios' creada exitosamente.")



# Paso 2: Preprocesamiento y Limpieza de Datos

### 🎯 Objetivo de la Actividad
En esta sección, prepararemos todos los datos para que puedan ser utilizados por el modelo de Machine Learning. Realizaremos una limpieza exhaustiva para manejar valores faltantes, corregir inconsistencias y asegurar que todas las variables estén en formato numérico.

### 📋 Tareas de este capítulo:
1.  **Limpieza Global de Datos**: Aplicaremos un conjunto de reglas para limpiar todas las variables del dataset en un solo paso.
2.  **Selección Final de Variables**: Definiremos los conjuntos `X` (predictoras) y `y` (objetivo) a partir de los datos ya limpios.



### 2.1. Limpieza y Preparación Exhaustiva de Datos

A continuación, ejecutaremos una celda de código que realiza todas las tareas de limpieza necesarias. Aunque el código es extenso, las acciones principales son:
- **Eliminar columnas no informativas**: Se descartan identificadores únicos, metadatos y variables de texto libre.
- **Evitar Fuga de Datos**: Se elimina la variable `contagiados`, ya que se usó para crear nuestro objetivo.
- **Corregir datos anómalos**: Se ajustan valores ilógicos, como montos negativos.
- **Manejar valores faltantes**: La mayoría de las variables de encuesta (`P66`, `P67`, etc.) tienen muchos datos faltantes. Asumiremos que un dato faltante significa "No" o "Cero" y los rellenaremos con `0`.
- **Estandarizar la codificación**: Nos aseguraremos de que todas las variables de tipo "Sí/No" usen un formato binario consistente (1 para "Sí", 0 para "No").
- **Convertir todo a numérico**: Se asegura que todas las columnas finales sean de tipo numérico.


In [ ]:

# Crear una copia del DataFrame para no alterar el original.
df_clean = df.copy()

# --- 1. Eliminar columnas no informativas o problemáticas ---
# Se eliminan identificadores, metadatos, texto libre y la variable de fuga de datos.
cols_to_drop = [
    'UBIGEO', 'DEPARTAMENTO', 'PROVINCIA', 'DISTRITO', # Identificadores
    'VFI_P66', 'VFI_P67', 'VFI_P68', '_merge',       # Metadatos de la encuesta
    'P67_11_O', 'P68_8_O',                           # Columnas de texto "Otro"
    'contagiados'                                   # ¡Variable de fuga de datos!
]
df_clean = df_clean.drop(columns=cols_to_drop)
print(f"✅ Se eliminaron {len(cols_to_drop)} columnas no informativas.")


# --- 2. Corregir y transformar variables específicas ---
# Corregir valores negativos en MONTO_GIRADO.
df_clean.loc[df_clean['MONTO_GIRADO'] < 0, 'MONTO_GIRADO'] = 0

# Convertir 'mes' y 'year' de texto a número.
df_clean[['mes', 'year']] = df_clean[['mes', 'year']].apply(pd.to_numeric, errors='coerce')
print("✅ Se corrigió MONTO_GIRADO y se convirtieron 'mes' y 'year' a numérico.")


# --- 3. Manejo de Valores Faltantes (Imputación) ---
# En las variables de encuesta (PXX), los valores faltantes se imputarán con 0.
# Esto asume que si no hay respuesta, la actividad no se realizó.
p_cols = [col for col in df_clean.columns if col.startswith('P')]
df_clean[p_cols] = df_clean[p_cols].fillna(0)
print(f"✅ Se imputaron valores faltantes con 0 en {len(p_cols)} columnas de encuesta.")


# --- 4. Estandarización de la Codificación Binaria ---
# Algunas preguntas usan 2 para "No". Las convertimos a 0 para mantener el estándar 0=No, 1=Sí.
p66_recode_2_to_0 = ['P66_1', 'P66_2', 'P66_3', 'P66_4', 'P66_5', 'P66_6', 'P66_7', 'P66_8', 'P66_9', 'P66_10']
for col in p66_recode_2_to_0:
    if col in df_clean.columns:
        df_clean[col] = df_clean[col].replace(2, 0)

# Otras preguntas usan valores > 1 para "Sí". Las convertimos a 1.
p_recode_gt0_to_1 = [col for col in df_clean.columns if col.startswith(('P67_', 'P68_'))]
for col in p_recode_gt0_to_1:
    df_clean[col] = df_clean[col].apply(lambda x: 1 if x > 0 else 0)
print("✅ Se estandarizó la codificación de variables binarias (0=No, 1=Sí).")


# --- 5. Verificación Final ---
# Comprobar que todas las columnas son numéricas y no hay valores nulos.
print("\n--- Verificación Final del DataFrame Limpio ---")
df_clean.info()

print("\n✅ ¡El preprocesamiento y la limpieza de datos han finalizado!")



### 2.2. Selección Final de Conjuntos de Datos (X, y)

Con el DataFrame `df_clean` completamente procesado, el último paso es separar nuestras variables predictoras (`X`) de nuestra variable objetivo (`y`).


In [ ]:

# 1. Crear el vector objetivo 'y', codificando "ALTO" como 1 y "BAJO" como 0.
y = df_clean['nivel_contagios'].apply(lambda nivel: 1 if nivel == 'ALTO' else 0)

# 2. Crear la matriz de características 'X' eliminando la columna objetivo.
X = df_clean.drop(columns=['nivel_contagios'])

print("✅ Conjuntos de datos X e y creados a partir de los datos limpios.")
print(f"Forma de X (predictoras): {X.shape}")
print(f"Forma de y (objetivo): {y.shape}")

print("\n--- Primeras 5 filas de X (variables predictoras) ---")
print(X.head())



### 🎓 Resumen del Paso 2

**Logros de este capítulo:**
- **Datos Listos para el Modelo**: Hemos procesado todas las variables disponibles, creando un conjunto de datos robusto y limpio.
- **Estrategia de Limpieza Aplicada**: Se manejaron valores faltantes, se corrigieron datos y se estandarizó la codificación de manera eficiente.
- **Conjuntos Finales Definidos**: Tenemos nuestros datos `X` (con 62 variables predictoras) y `y` listos para la siguiente fase: entrenamiento y evaluación del modelo.
